In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from response.feature import Feature

In [3]:
F_path = "../X_y_data/response2/"
F_name = "Classify_F2.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response2/Classify_F2.pickle


In [8]:
model_path = "../models/response2/"
model_name = "Classify_M2.pickle"
modelM = DataManager(model_path)
clf = modelM.load_data(model_name)

success load : ../models/response2/Classify_M2.pickle


In [9]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [30]:
# 質問系かどうかチェックする
usr_sys = []
y = []
errors = ["Ignore question", "Ignore offer", "Ignore proposal", "Ignore greeting"]
for conv in convs:
    for i, ut in enumerate( conv ) :
        if ut.is_system() :
            if ut.is_exist_error():
                usr_sys.append( [conv[i-1].utt, ut.utt] )
                if ut.is_error_included(errors):
                    y.append(1)
                else:
                    y.append(0)


In [31]:
from tqdm import tqdm
print(F.feature_num)
usr = []
for i, u_s in enumerate( tqdm(usr_sys) ):
    x = F.featurization(u_s[0])
    usr.append(x)
usr_ = np.array(usr)

  0%|          | 0/1386 [00:00<?, ?it/s]

56880


100%|██████████| 1386/1386 [08:03<00:00,  2.86it/s]


In [32]:
question_pred = clf.predict(usr_)
# np.count_nonzero(question_pred) : 643

In [33]:
from sentence_transformers import SentenceTransformer
bert_path = "../../corpus/sbert_ignore"
sbert = SentenceTransformer(bert_path)

In [34]:
smodel_path = "../models/response2/"
smodel_name = "Classify_M_sbert.pickle"
smodelM = DataManager(smodel_path)

sclf = smodelM.load_data(smodel_name)

success load : ../models/response2/Classify_M_sbert.pickle


In [35]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [36]:

question_X = []
for u_s, qp in zip(usr_sys, question_pred):
    if qp==1:
        question_X.append(u_s)

In [37]:
X_sbert = [ text2feature(x) for x in question_X]
sbeert_pred = sclf.predict(X_sbert)

In [40]:
count = 0
y_pred = []
for p in question_pred:
    if p==1:
        if sbeert_pred[count] <2:
            y_pred.append(0)
        else:
            y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

In [41]:
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[878 154]
 [ 82 272]]
accuracy =  0.8297258297258298
precision =  0.6384976525821596
recall =  0.768361581920904
f1 score =  0.6974358974358975
